In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx

In [16]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

In [3]:
df_glu = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df_glu = thermo_df(df_glu)
df_glu.shape

(4593, 3)

In [4]:
df = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/Excels/tRNA_Phe_0724.xlsx')
# df_phe = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/BaoLab/FirstBatch/Excels/S2to5combined_20ul_40min.xlsx')
df = thermo_df(df)
df.shape
# plotly_zone(df)

(5933, 3)

# Cut 2 times

In [211]:
ms_idxs = list()
for idx, row in dftop.iterrows():
    ms_idxs.append((row.Mass, idx))
len(ms_idxs)

1000

In [212]:
ms_idxs[0]

(23904.2359311045, 1994)

In [213]:
from itertools import combinations

In [214]:
sums = list()
idxs = list()
for comb in combinations(ms_idxs, 3):
    masssum = sum([x[0] for x in comb]) - 18.0106 * 2
    sums.append(masssum)
    idx = [x[1] for x in comb]
    idxs.append(idx)
#     break

# len(sums), len(idxs)
print('done')

done


In [215]:
df_sums = pd.Series(sums)
df_idxs = pd.DataFrame(idxs)

In [264]:
import pickle
with open('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/PaperData/df_sums.pkl', 'wb') as f:
    pickle.dump(df_sums, f)

with open('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/Data/PaperData/df_idxs.pkl', 'wb') as f:
    pikcle.dump(df_idxs, f)

NameError: name 'pkcle' is not defined

In [251]:
# ms = 24252.31 #25334.57 #25005.52 #24267.31
# masses = [23904.24, 23599.19, 24233.28] # Asp
masses = [24287.35, 24460.38] # Ala, Val
# masses = [24460.38] # Val
ms = masses[0]
df_chosen = df_sums[(df_sums>=ms-0.05)&(df_sums<ms+0.05)]
len(df_idxs.loc[df_chosen.index])

748

In [252]:
df_idxs_chosen = df_idxs.loc[df_chosen.index]
df_idxs_chosen.shape

(748, 3)

In [253]:
np_idxs = np.array(df_idxs_chosen)
np_uni_idxs = np.unique(np_idxs)

In [254]:
df_tmp = dftop.loc[np_uni_idxs]
plotly_zone(df_tmp)
df_tmp.shape
df_tmp.to_excel('/Users/bryan/Downloads/cut2times.xlsx')

In [5]:
df_phe_5p = pd.read_excel('/Users/bryan/Documents/BioPharmaFinder/ZhangLab/MassSum/GraphsOutput/Graphs_1_24581/S3_gap_fill_5p.xlsx')
df_phe_5p.shape

(72, 6)

In [44]:
full_mass = 21839.9673 #23088.1066 #23393.1479 #24581.3
df_fd_phe = computational_data_seperation(df_phe_5p, df, full_mass, ignore_endpoints=True)
# df_fd_phe = computational_data_seperation()
# plotly_basecalling(*df_fd_phe)
plotly_zones(*df_fd_phe, names=["5'dots", "mid dots"])

full_mass 21839.9673 sum_value 21858.0


In [17]:
bcr = base_calling_random(df_fd_phe[1])
plotly_basecalling(*bcr, y='RT', annotate=False)

mass_x (25,) mass_y (25,)


In [18]:
bcr[1]

[(3551.48805702449, 3246.44796876393, 'C'),
 (3856.53103266395, 3551.48805702449, 'C'),
 (5155.70971690984, 4850.65541184426, 'C'),
 (1938.23864704917, 1609.20237446425, 'A'),
 (1938.2545465196, 1609.20237446425, 'A'),
 (4850.65541184426, 4505.62102857279, 'G'),
 (6126.81252380505, 5781.77568553057, 'G'),
 (6777.89625965983, 6432.85982460842, 'G'),
 (9392.25527223929, 9047.20742812289, 'G'),
 (5461.73919096891, 5155.70971690984, 'U'),
 (6432.85982460842, 6126.81252380505, 'U'),
 (17834.4819753261, 17526.392532178, 'D'),
 (5781.77568553057, 5461.73919096891, 'u')]

# Homology Search & Group

In [40]:
def homology_groups(df):
    """ given ~25k area dots, doing homology search
        return a list of group, each group represents a tRNA specy and its isoforms 
    """
    def create_graph(bcr):
        G = nx.Graph()
        for x in bcr[1]:
            G.add_edge(x[0], x[1], name=x[2])

        cc = list(nx.connected_components(G))
        subgraphs = [G.subgraph(x) for x in cc]
        subgraphs.sort(key=len, reverse=True)
        return subgraphs
    
    bcr = base_calling_random(df, homology=True)
    subgraphs = create_graph(bcr)
    subgraphs_edges = [ [(x[0], x[1], x[2].get('name')) for x in sg.edges(data=True)] for sg in subgraphs]
    subgraphs_edges = [sorted(sg_edges, key=lambda x: x[0]) for sg_edges in subgraphs_edges]
    subgraphs_nodes = [list(sg.nodes) for sg in subgraphs]
    
    node_edge_pairs = list()
    for group in range(len(subgraphs)):
        df_nodes = df[df.Mass.isin(subgraphs_nodes[group])]
        edges = subgraphs_edges[group]
        node_edge_pairs.append((df_nodes, edges))
    
    return node_edge_pairs

In [41]:
groups = homology_groups(df_25k)
groups[0]
# plotly_basecalling(*groups[0], y='Vol', annotate=False)

mass_x (27,) mass_y (27,)


(              Mass         RT          Vol
 5812  23722.165528  16.014392    845895.83
 5819  24027.244899  16.049153   3460015.41
 5831  24238.334749  15.945038    511799.89
 5833  24252.311092  15.979698  67449456.40
 5835  24268.303777  16.014392   5195981.25
 5836  24280.305501  15.979698   7527536.79
 5837  24294.304910  15.979698    517550.21
 5845  24332.311220  16.014392   2428455.65
 5857  24581.380920  16.014392  41918736.85
 5861  24597.353308  16.014392   3340775.80,
 [(23722.1655280383, 24027.2448987364, 'C'),
  (24027.2448987364, 24252.3110919712, 'Ci'),
  (24238.3347493023, 24252.3110919712, 'Methyl'),
  (24268.3037773893, 24597.3533080084, 'A'),
  (24268.3037773893, 24252.3110919712, 'Ox'),
  (24280.3055005283, 24252.3110919712, '2Methyl'),
  (24294.304910419, 24280.3055005283, 'Methyl'),
  (24332.3112203026, 24027.2448987364, 'C'),
  (24581.380920434, 24252.3110919712, 'A'),
  (24581.380920434, 24332.3112203026, 'Ai'),
  (24581.380920434, 24597.3533080084, 'Ox')])

In [27]:
df_25k = df[(df.Mass > 2.3E4) & (df.Mass < 2.6E4) & (df.Vol > 5E5)]
bcr = base_calling_random(df_25k, homology=True)
# plotly_basecalling(*bcr, y='Vol')

mass_x (27,) mass_y (27,)


In [20]:
G = nx.Graph()
for x in bcr[1]:
    G.add_edge(x[0], x[1], name=x[2])

cc = list(nx.connected_components(G))
subgraphs = [G.subgraph(x) for x in cc]
subgraphs.sort(key=len, reverse=True)
len(subgraphs)

5

In [21]:
subgraphs_edges = [ [(x[0], x[1], x[2].get('name')) for x in sg.edges(data=True)] for sg in subgraphs]
subgraphs_nodes = [list(sg.nodes) for sg in subgraphs]

subgraphs_edges = [sorted(sg_edges, key=lambda x: x[0]) for sg_edges in subgraphs_edges]

In [30]:
idx = 1
group = idx - 1
df_nodes = df[df.Mass.isin(subgraphs_nodes[group])]
df_edges = subgraphs_edges[group]
plotly_basecalling(df_nodes, df_edges, y='Vol', annotate=False)
df_nodes.Mass, df_edges

(5812    23722.165528
 5819    24027.244899
 5831    24238.334749
 5833    24252.311092
 5835    24268.303777
 5836    24280.305501
 5837    24294.304910
 5845    24332.311220
 5857    24581.380920
 5861    24597.353308
 Name: Mass, dtype: float64,
 [(23722.1655280383, 24027.2448987364, 'C'),
  (24027.2448987364, 24252.3110919712, 'Ci'),
  (24238.3347493023, 24252.3110919712, 'Methyl'),
  (24268.3037773893, 24597.3533080084, 'A'),
  (24268.3037773893, 24252.3110919712, 'Ox'),
  (24280.3055005283, 24252.3110919712, '2Methyl'),
  (24294.304910419, 24280.3055005283, 'Methyl'),
  (24332.3112203026, 24027.2448987364, 'C'),
  (24581.380920434, 24252.3110919712, 'A'),
  (24581.380920434, 24332.3112203026, 'Ai'),
  (24581.380920434, 24597.3533080084, 'Ox')])

In [24]:
for idx, sg in enumerate(subgraphs):
    edges = list(sg.edges(data=True))
    edges = [(x[0], x[1], x[2].get('name')) for x in edges]
    edges.sort(key=lambda x:x[0], reverse=True)
    print(f'Group{idx+1}\n', edges)

Group1
 [(24581.380920434, 24252.3110919712, 'A'), (24581.380920434, 24332.3112203026, 'Ai'), (24581.380920434, 24597.3533080084, 'Ox'), (24332.3112203026, 24027.2448987364, 'C'), (24294.304910419, 24280.3055005283, 'Methyl'), (24280.3055005283, 24252.3110919712, '2Methyl'), (24268.3037773893, 24597.3533080084, 'A'), (24268.3037773893, 24252.3110919712, 'Ox'), (24238.3347493023, 24252.3110919712, 'Methyl'), (24027.2448987364, 24252.3110919712, 'Ci'), (23722.1655280383, 24027.2448987364, 'C')]
Group2
 [(24610.4226708103, 24596.3566028188, 'Methyl'), (24267.3064600734, 24596.3566028188, 'A'), (24267.3064600734, 24239.2642630857, '2Methyl')]
Group3
 [(24416.2863040009, 24087.242612065, 'A'), (24087.242612065, 24103.2693164027, 'Ox')]
Group4
 [(24116.2673770374, 24445.3466810474, 'A')]
Group5
 [(25005.5183393632, 25334.5687566135, 'A')]
